In [291]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [292]:
# Load user profiles
df_user_profiles = pd.read_csv('Data//user_profiles_edited.csv')

In [293]:
df_user_profiles

,User_ID,User_Name,Occupasion,Age,Interests,Country
0,1,johndoe,Software Engineer,25,"Adventure, Nature",USA
1,2,janedoe,Teacher,30,"Culture, History",Canada
2,3,alice,Student,22,"Beaches, Relaxation",UK
3,4,bob,Photographer,28,"Wildlife, Adventure",Australia
4,5,charlie,Environmental Scientist,35,"Nature, Wildlife",India
5,6,dave,Historian,40,"Cultural, Historical",USA
6,7,emma,Travel Blogger,27,"Adventure, Hiking",Canada
7,8,frank,Photographer,32,Wildlife,Germaney
8,9,Rajapaksha,Lawyer,43,"Cultural Exploration, Networking Events, Relax...",Sri Lanka


In [294]:
# Load the user_reviews
df_user_reviews = pd.read_csv('Data\\user_reviews_edited.csv')

In [295]:
df_user_reviews

,Review ID,User ID,Location,Rating,Review
0,1,1,Yala,5,Amazing wildlife experience
1,2,1,Horton Plains,4,Beautiful hiking trails
2,3,2,Kandy,4,Rich history and culture
3,4,2,Sigiriya,5,Stunning ancient rock fortress
4,5,3,Hikkaduwa,3,Nice beaches but can get crowded
5,6,3,Unawatuna,4,Lovely beach and great food
6,7,4,Yala,5,Incredible safari and animal sightings
7,8,4,Nuwara Eliya,4,Beautiful scenery and tea plantations
8,9,5,Sinharaja,5,A great place for nature lovers
9,10,5,Pinnawala,4,Interesting elephant orphanage


In [296]:
"""
# Recommend users with similar interests
def recommend_users(interests):
    # Convert input interests to a set for easy comparison
    user_interest_set = set(interests)

    # List to hold recommended users
    recommendations = []

    # Loop through user_profiles to find similar interests
    for index, profile in user_profiles.iterrows():  # Use iterrows to iterate over DataFrame rows
        common_interests = user_interest_set.intersection(set(profile['Interests'].split(',')))  # Use profile to access interests
        if common_interests and profile["User_Name"] != interests:  # Avoid recommending the same user
            recommendations.append((profile["User_Name"], len(common_interests)))  # Use profile to access User_Name

    # Sort by number of common interests (descending) and take the top 5
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return [user for user, _ in recommendations[:5]]

# Example usage
input_interests = ["Beaches", "Relaxation"]  # User input interests
recommended_users = recommend_users(input_interests)

print("Recommended users with similar interests:")
for user in recommended_users:  # Change to recommended_users
    print(user)
"""

'\n# Recommend users with similar interests\ndef recommend_users(interests):\n    # Convert input interests to a set for easy comparison\n    user_interest_set = set(interests)\n\n    # List to hold recommended users\n    recommendations = []\n\n    # Loop through user_profiles to find similar interests\n    for index, profile in user_profiles.iterrows():  # Use iterrows to iterate over DataFrame rows\n        common_interests = user_interest_set.intersection(set(profile[\'Interests\'].split(\',\')))  # Use profile to access interests\n        if common_interests and profile["User_Name"] != interests:  # Avoid recommending the same user\n            recommendations.append((profile["User_Name"], len(common_interests)))  # Use profile to access User_Name\n\n    # Sort by number of common interests (descending) and take the top 5\n    recommendations.sort(key=lambda x: x[1], reverse=True)\n    return [user for user, _ in recommendations[:5]]\n\n# Example usage\ninput_interests = ["Beaches

In [297]:
# Merge Dataset

In [298]:
print(df_user_profiles.columns)
print(df_user_reviews.columns)

Index(['User_ID', 'User_Name', 'Occupasion', 'Age', 'Interests', 'Country'], dtype='object')
Index(['Review ID', 'User ID', 'Location', 'Rating', 'Review '], dtype='object')


In [299]:
df_user_reviews.rename(columns={'User ID': 'User_ID'}, inplace=True)

In [300]:
df_merged = pd.merge(df_user_profiles, df_user_reviews, on="User_ID", how="inner")

In [301]:
df_merged

,User_ID,User_Name,Occupasion,Age,Interests,Country,Review ID,Location,Rating,Review
0,1,johndoe,Software Engineer,25,"Adventure, Nature",USA,1,Yala,5,Amazing wildlife experience
1,1,johndoe,Software Engineer,25,"Adventure, Nature",USA,2,Horton Plains,4,Beautiful hiking trails
2,2,janedoe,Teacher,30,"Culture, History",Canada,3,Kandy,4,Rich history and culture
3,2,janedoe,Teacher,30,"Culture, History",Canada,4,Sigiriya,5,Stunning ancient rock fortress
4,3,alice,Student,22,"Beaches, Relaxation",UK,5,Hikkaduwa,3,Nice beaches but can get crowded
5,3,alice,Student,22,"Beaches, Relaxation",UK,6,Unawatuna,4,Lovely beach and great food
6,4,bob,Photographer,28,"Wildlife, Adventure",Australia,7,Yala,5,Incredible safari and animal sightings
7,4,bob,Photographer,28,"Wildlife, Adventure",Australia,8,Nuwara Eliya,4,Beautiful scenery and tea plantations
8,5,charlie,Environmental Scientist,35,"Nature, Wildlife",India,9,Sinharaja,5,A great place for nature lovers
9,5,charlie,Environmental Scientist,35,"Nature, Wildlife",India,10,Pinnawala,4,Interesting elephant orphanage


In [302]:
df_merged.columns

Index(['User_ID', 'User_Name', 'Occupasion', 'Age', 'Interests', 'Country',
       'Review ID', 'Location', 'Rating', 'Review '],
      dtype='object')

In [303]:
# create a combined feature string for each user
df_merged['combined_features'] = (df_merged['Occupasion'] + ' ' + df_merged['Age'].astype(str) + ' ' + df_merged['Interests'] + ' ' + df_merged['Country'])

In [304]:
# Create a combined feature string for input user
input_user_combined = (str(df_merged['Occupasion']) + ' ' + str(df_merged['Age']) + ' ' + str(df_merged['Interests']) + ' ' + str(df_merged['Country']))

In [305]:
df_merged.columns

Index(['User_ID', 'User_Name', 'Occupasion', 'Age', 'Interests', 'Country',
       'Review ID', 'Location', 'Rating', 'Review ', 'combined_features'],
      dtype='object')

In [306]:
# Recommend similar users
def recommend_similar_users(input_users):
    pass
    

In [307]:
# append the input users combined features to the df_merged Data Frame
combined_features = df_merged['combined_features'].tolist() + [input_user_combined]

In [308]:
combined_features

['Software Engineer 25 Adventure, Nature USA',
 'Software Engineer 25 Adventure, Nature USA',
 'Teacher 30 Culture, History Canada',
 'Teacher 30 Culture, History Canada',
 'Student 22 Beaches, Relaxation  UK',
 'Student 22 Beaches, Relaxation  UK',
 'Photographer 28 Wildlife, Adventure Australia',
 'Photographer 28 Wildlife, Adventure Australia',
 'Environmental Scientist 35 Nature, Wildlife India',
 'Environmental Scientist 35 Nature, Wildlife India',
 '0          Software Engineer\n1          Software Engineer\n2                    Teacher\n3                    Teacher\n4                    Student\n5                    Student\n6               Photographer\n7               Photographer\n8    Environmental Scientist\n9    Environmental Scientist\nName: Occupasion, dtype: object 0    25\n1    25\n2    30\n3    30\n4    22\n5    22\n6    28\n7    28\n8    35\n9    35\nName: Age, dtype: int64 0       Adventure, Nature\n1       Adventure, Nature\n2        Culture, History\n3        Cult

In [309]:
type(combined_features)

list

In [310]:
# vectorize the combined features
vectorizer = CountVectorizer()
combined_features_matrix = vectorizer.fit_transform(combined_features)

In [314]:
combined_features_matrix

<11x32 sparse matrix of type '<class 'numpy.int64'>'
	with 86 stored elements in Compressed Sparse Row format>

In [317]:
combined_features_matrix.toarray()

array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0,

In [330]:
# Determine the similarity
similarity = cosine_similarity(combined_features)

NameError: name 'combined_features_matrix' is not defined